In [ ]:
### Libraries

In [ ]:
import os
from io import BytesIO
import requests
from datetime import datetime,timedelta
import warnings
import xarray as xr
import io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
import pandas as pd
import matplotlib.patheffects as path_effects
from matplotlib.colors import ListedColormap
from matplotlib.colorbar import ColorbarBase
import matplotlib.colors as mcolors
import image_functions as imagef
import plotting_functions as plotf
import helper_dicts as hdict
import helper_functions as helper
%load_ext autoreload
%autoreload 2

In [ ]:
layerlinks = {
            'Monitor Tools': {
                'OISST SST 7-Day Anomaly': 'https://www.cpc.ncep.noaa.gov/products/international/oisst/oisst_7day_pac_anom.gif',
                'OISST SST 7-Day Anomaly Difference': 'https://www.cpc.ncep.noaa.gov/products/international/oisst/oisst_7day_pac_diff.gif',
                'CMORPH Time Series Points': 'https://www.cpc.ncep.noaa.gov/products/international/zz_CP_Islands/zz_CMORPH_BLD_2nd/CMORPH_BLD_Islands_180day.shtml',
                'CMORPH 90 Day Precip Anomaly': 'https://www.cpc.ncep.noaa.gov/products/international/drought/precip/img/Anom_90day_PacificIslands.png',
                'CMORPH 90 Day Precip Percent': 'https://www.cpc.ncep.noaa.gov/products/international/drought/precip/img/Perc_90day_PacificIslands.png',
                'CMORPH 3-month SPI': 'https://www.cpc.ncep.noaa.gov/products/international/dm/daily/Pacific_CMORPH_SPI_3mo.png',
                'CMORPH 3-month SPEI': 'https://www.cpc.ncep.noaa.gov/products/international/dm/daily/Pacific_CMORPH_SPEI_3mo.png',
                'CMORPH 30 Day Precip Anomaly': 'https://www.cpc.ncep.noaa.gov/products/international/drought/precip/img/Anom_30day_PacificIslands.png',
                'CMORPH 30 Day Precip Percent': 'https://www.cpc.ncep.noaa.gov/products/international/drought/precip/img/Perc_30day_PacificIslands.png',
                'CMORPH 1-month SPI': 'https://www.cpc.ncep.noaa.gov/products/international/dm/daily/Pacific_CMORPH_SPI_1mo.png',
                'CMORPH 1-month SPEI': 'https://www.cpc.ncep.noaa.gov/products/international/dm/daily/Pacific_CMORPH_SPEI_1mo.png',
                'Leaky Bucket Model 1-month Surface Runoff Index': 'https://www.cpc.ncep.noaa.gov/products/people/lxu/leaky/smp/SRI1_PacificIslands.png',
                'CMORPH 7 Day Precip Total': 'https://www.cpc.ncep.noaa.gov/products/international/drought/precip/img/Total_7day_PacificIslands.png',
# 'https://github.ncep.noaa.gov/katie-kowal/pac_isl_maps/blob/main/weeklycmorph.png?raw=true',   
               # // https://github.com/katiemkowal/pac_isl_maps/blob/main/weeklycmorph.png?raw=true
                'CMORPH 7 Day Precip Anomaly': 'https://www.cpc.ncep.noaa.gov/products/international/drought/precip/img/Anom_7day_PacificIslands.png',
                'CMORPH 7 Day SPI': 'https://www.cpc.ncep.noaa.gov/products/international/dm/daily/Pacific_CMORPH_SPI_1wk.png',
                'CDAS 7 Day Tmax':  'https://ftp.cpc.ncep.noaa.gov/rshukla/CDAS/CDAS_7dy_Tmax_Ave_CPI.png',
                'CDAS 7 Day Tmax Anomaly': 'https://ftp.cpc.ncep.noaa.gov/rshukla/CDAS/CDAS_7dy_Tmax_anom_CPI.png',
                'VHI 1km': 'https://www.ncei.noaa.gov/pub/data/nidis/tile/noaa-vhi-1km/{z}/{x}/{y}.png',
                'NASA ESI': 'https://www.ncei.noaa.gov/pub/data/nidis/tile/nasa-esi-5km-4wk/{z}/{x}/{y}.png'
            },
            'Forecast Tools': {
                'GEFS Week 1 Precip Total Forecast':  'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week1_figures/gefs_week1_precipt.png',
                'GEFS Week 1 Precip Anomaly Forecast': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week1_figures/gefs_week1_precip.png',
                'GEFS Week 1 Precip Exceedance Prob (50 mm)': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week1_figures/gefs_week1_prob_50.png',
                'GEFS Week 1 Precip Exceedance Prob (100 mm)': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week1_figures/gefs_week1_prob_100.png',
                'GEFS Week 1 Raw Precip Tercile Prob': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_calib/fig_dir/gefs_week_1_raw.png',
                'GEFS Week 1 Consolidated Precip Tercile Prob': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_calib/fig_dir/gefs_week_1_cons.png',
                'GEFS Week 1 Temp Total Forecast': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week1_figures/gefs_week1_t2mt.png',
                'GEFS Week 1 Temp Anomaly Forecast': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week1_figures/gefs_week1_t2m.png',
                'Leaky Bucket Model Evaporative Stress Index Week 1 Forecast': 'https://www.cpc.ncep.noaa.gov/products/people/lxu/leaky/smp/ESI_Week1_PacificIslands.png',
                'Leaky Bucket Model Soil Moisture Percentile Week 1 Forecast':'https://www.cpc.ncep.noaa.gov/products/people/lxu/leaky/smp/SMP_Week1_PacificIslands.png',
                'Leaky Bucket Model SPI Week 1 Forecast': 'https://www.cpc.ncep.noaa.gov/products/people/lxu/leaky/smp/SPI3_Week1_PacificIslands.png',
                'GEFS Week 2 Precip Total':'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week2_figures/gefs_week2_precipt.png',
                'GEFS Week 2 Precip Anomaly': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week2_figures/gefs_week2_precip.png',
                'GEFS Week 2 Temp Total Forecast': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week2_figures/gefs_week2_t2mt.png',
                'GEFS Week 2 Temp Anomaly Forecast': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week2_figures/gefs_week2_t2m.png',
                'GEFS Week 2 Precip Exceedance Prob (50 mm)': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week2_figures/gefs_week2_prob_50.png',
                'GEFS Week 2 Precip Exceedance Prob (100 mm)': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_raw/gefs_week2_figures/gefs_week2_prob_100.png',
                'GEFS Week 2 Raw Precip Tercile Prob': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_calib/fig_dir/gefs_week_2_raw.png',
                'GEFS Week 2 Consolidated Precip Tercile Prob': 'https://ftp.cpc.ncep.noaa.gov/International/PREPARE_Pacific/subseasonal/gefs_calib/fig_dir/gefs_week_2_cons.png',
                'NMME 1 Month Precip Anomaly Forecast': 'https://www.cpc.ncep.noaa.gov/products/international/drought/nmme/Multi_Model_Ensemble_Panom_L1_PacificIslands.png',
                'NMME 1 Month SPI Forecast': 'https://www.cpc.ncep.noaa.gov/products/international/drought/nmme/Multi_Model_Ensemble_SPI3_L1_PacificIslands.png'
            }
        }

In [ ]:
plotf.plot_original_image(spei_path.crop((150, 650, 900, 660)))

In [ ]:
spei_path = Image.open(BytesIO(requests.get(layerlinks['Monitor Tools']['CMORPH 1-month SPEI']).content))
spei_colorbar = imagef.convert_image_to_array(spei_path, (150, 650, 900, 660))
spei_colors = imagef.calc_unique_colors(spei_colorbar)
spei_colors = {imagef.closest_color_category(color, hdict.color_categories): tuple(color.tolist()) for color in spei_colors}
pac_cropw = (45,65, 676, 555) #65
pac_crope = (676,65, 1004, 555)
pac_coordsw = (132, 180, 9, -22) #is 10 
pac_coordse = (180, 205, 9, -22)
spei_pacw = imagef.convert_image_to_array(spei_path, pac_cropw)
spei_pace = imagef.convert_image_to_array(spei_path, pac_crope)
spei_pacw = imagef.convert_image_to_coordinates(spei_pacw, pac_coordsw[0], pac_coordsw[1], pac_coordsw[2], pac_coordsw[3])
spei_pace = imagef.convert_image_to_coordinates(spei_pace, pac_coordse[0], pac_coordse[1], pac_coordse[2], pac_coordse[3])
spei_pac = pd.concat([spei_pacw, spei_pace], ignore_index = True)
df_spei_pac = imagef.convert_df_to_reduced_gdf(spei_pac, 1, 1)
df_spei_pac = imagef.convert_color(df_spei_pac)
df_spei_pac.x = (df_spei_pac.x + 360)%360

In [ ]:
df_spei_pac.plot(color = df_spei_pac['color'])